In [53]:
import pandas as pd

db = pd.read_csv("../lapacke_db/lapacke.csv", index_col=0)
db = db[~ db['arg_type'].map(lambda x: ('LAPACK' in x))]

wrapped = set(db['fct_name'].map(lambda x: x.replace("LAPACKE_", "")).unique())

import glob

prefix = "../../../lapack/SRC/"
file_list = glob.glob(prefix + "*.f")

file_names = set(map(lambda x: x.replace(prefix, "").replace(".f",""), file_list))

In [54]:
excluded_files = {f for f in file_names if not f in wrapped}
print("Files that should be not wrapped:")
print(excluded_files)

missing_files = {f for f in wrapped if not f in file_names}
print("Files missing:")
print(missing_files)

file_names = {f for f in file_names if f not in excluded_files and f not in missing_files}
file_list  = [prefix + f + ".f" for f in file_names] 

print(file_list[-1])

Files that should be not wrapped:
{'slarrd', 'zsytf2_rk', 'chetrd_2stage', 'clarzb', 'zhetf2_rook', 'ssytri2', 'dlarrk', 'cunhr_col', 'cspr', 'cheevx_2stage', 'clasyf_rk', 'cgemqr', 'sorgr2', 'clarft', 'csytrs_rook', 'dla_syrcond', 'dlasdt', 'dlaqr1', 'chetri_3', 'dgeqrt3', 'zsytrs2', 'sgeql2', 'clapll', 'clahef_aa', 'sorbdb6', 'slansb', 'zlarfg', 'clalsa', 'dlaqz4', 'zpstf2', 'zhetd2', 'slarfg', 'dorbdb', 'ssytrf_rk', 'cgsvj0', 'slantp', 'dlaed7', 'dlalsa', 'dorbdb5', 'zlaqge', 'clanhb', 'ssytrd_sy2sb', 'slansf', 'dlasd2', 'sgelqt', 'slals0', 'dgetc2', 'zlatrd', 'slarrc', 'chetri_rook', 'slasd2', 'slauu2', 'dorbdb3', 'zgerfsx', 'dsytri2', 'dorg2l', 'sisnan', 'cgesc2', 'slapy3', 'sgsvj0', 'dlaqsb', 'dlaisnan', 'zlarfx', 'cheevr_2stage', 'dlarrf', 'clantb', 'chesv_rk', 'clarfb_gett', 'dorm22', 'dlamtsqr', 'dlaqtr', 'dorcsd2by1', 'zcposv', 'chetrd_he2hb', 'clascl2', 'chetf2_rk', 'zla_hercond_x', 'zlasyf', 'cla_gbrfsx_extended', 'sgelst', 'sgerq2', 'dlarrr', 'dsyconv', 'csyconvf_rook', 'z

#### Extract the types of variables from each file

In [102]:
import re

def local_print(txt):
    #print(txt)
    return

def extract_between(file_path, prefix, suffix):
    local_print("Reading file " + file_path)
    extracted_content = []
    is_extracting = False

    with open(file_path, 'r') as file:
        for line in file:
            if 'implicit' in line.lower():
                continue
            #print("reading " + line)
            if prefix in line:
                #print('start')
                is_extracting = True
                continue
            
            if suffix == line[0] and is_extracting:
                #print('end')
                is_extracting = False
                break
            
            if is_extracting:
                extracted_content.append(line.strip())
    
    merged_lines = []
    for line in extracted_content:
        if "$" in line:
            line = line.replace("$", "").strip()
            merged_lines[-1] = merged_lines[-1] + line

            local_print("@@@@@@@@@@ MERGE RESULT " + merged_lines[-1])
        else:
            merged_lines.append(line)

    return merged_lines

def extract_declarations(file_path):
    scalar_prefix   = """*     .. Scalar"""
    array_prefix    = """*     .. Array"""
    suffix          = """*"""

    scalar = extract_between(file_path, scalar_prefix, suffix)
    array  = extract_between(file_path, array_prefix, suffix)

    if len(scalar[0]) == 1 or len(array[0]) == 1:
        local_print("Error reading " + file_path)

    return {"scalar": scalar, "array": array}


def extract_type(declaration):
    local_print(declaration)
    result = re.split("\s\s+", declaration)
    local_print(" Result is " + result[0])
    
    if len(result) == 1:
        local_print("Issue with " + result[0] + "\n attempting fix")
        # Split at the last space occuring before a comma
        modified_declaration = declaration.replace("( ", "").replace(" )", "")
        first_comma = modified_declaration.find(",")
        last_space_index = modified_declaration.rfind(' ', 0, first_comma)

        result = modified_declaration[0:last_space_index]
        return result

    else:
        t, _ = result
        return t

Scalars = set()
Arrays = set()

for f in file_list:
    for Group, Insertions in zip((Scalars, Arrays), extract_declarations(f).values()):
        Group.update(set(map(extract_type, Insertions)))

print("Printing results:\nScalar types")
print(Scalars)
print("Arrays types")
print(Arrays)

Printing results:
Scalar types
{'COMPLEX*16', 'REAL', 'INTEGER', 'CHARACTER*1', 'CHARACTER', 'COMPLEX', 'LOGICAL', 'DOUBLE PRECISION'}
Arrays types
{'COMPLEX*16', 'REAL', 'INTEGER', 'CHARACTER*1', 'COMPLEX', 'LOGICAL', 'DOUBLE PRECISION'}


In [35]:
import re

def extract_subroutines(file_path):
    subroutines = []
    
    with open(file_path, 'r') as file:
        content = file.read()
        
        # Extract subroutine blocks using regular expressions
        subroutine_blocks = re.findall(r"(?i)^\s*SUBROUTINE\s+(\w+)\s*\((.*?)\)", content, re.DOTALL | re.MULTILINE)
        
        # Process each subroutine block
        for subroutine_block in subroutine_blocks:
            subroutine_name = subroutine_block[0].lower()
            subroutine_params = subroutine_block[1].replace(" ","").replace("\n$", "").split(",")
         
            
            local_print(subroutine_name, subroutine_params)

    return subroutines

def extract_content_between(file_path, prefix, suffix):
    extracted_content = []
    
    with open(file_path, 'r') as file:
        is_extracting = False
        
        for line in file:
            if line.strip() == prefix:
                is_extracting = True
                continue
            
            if line.strip() == suffix:
                is_extracting = False
            
            if is_extracting:
                extracted_content.append(line.strip())
    
    return extracted_content
def extract_variables(file_path):
    
# Extract subroutines, names, and parameters
subroutines = extract_subroutines(file_list[-1])

# local_print the results
for subroutine_name, subroutine_params in subroutines:
    local_print(f"Subroutine: {subroutine_name}")
    local_print("Parameters:")
    for param_name, param_type in subroutine_params:
        local_print(f"\tName: {param_name}, Type: {param_type}")
    local_print()

cgbtrs ['TRANS', 'N', 'KL', 'KU', 'NRHS', 'AB', 'LDAB', 'IPIV', 'B', 'LDB', 'INFO']
